In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import abr_fam_utils as ut

In [ ]:
def select_first_V(row):
    row['v_call'] = row['v_call'].split('*')[0]
    return row
def select_first_J(row):
    row['j_call'] = row['j_call'].split('*')[0]
    return row


def import_clone_frames(directory, samples=[]):
    
    clones = dict()
    for file_name in os.listdir(directory):

        if '_abr_clones.tsv' in file_name: # Loop over the clone files
            
            sample_id = file_name.split('_')[0]+'_'+file_name.split('_')[1]
            
            if len(samples) > 0 and sample_id not in samples:
                continue
                
            clone_frame = pd.read_csv(directory+"/"+file_name,sep='\t')
            clone_frame = clone_frame[['duplicate_count', 'junction', 
                                       'v_call', 'j_call']]
            clone_frame = clone_frame[clone_frame['duplicate_count'] > 1]
            clone_frame = clone_frame.apply(select_first_V, axis=1)
            clone_frame = clone_frame.apply(select_first_J, axis=1)
            sample_id = file_name.split('_')[0]+'_'+file_name.split('_')[1]
            clones[sample_id] = clone_frame
            
    return clones

def import_clone_frame_pat(directory, pat, keep_seq=False):
    """
    Import all the abr family dataframes in a dir of a patient
    """
    samples = []
    for file_name in os.listdir(directory):
        if file_name.split('_')[1][:3] == 'day':
            p = int(file_name.split('_')[0][3:])
            if p == int(pat):
                samples.append('pat' + str(p)+'_day'+file_name.split('_')[1][3:])
    return import_clone_frames(directory, samples)

In [ ]:
pat = '8'
clones = import_clone_frame_pat('../abr_clones', '8')

ids = []
frames = []
for id_, f in clones.items():
    ids.append(id_)
    frames.append(f)

In [ ]:
ab_fam_gen = ut.abr_families_gen(frames, 'duplicate_count', 'junction', 'v_call', 'j_call', False)
print(len(ab_fam_gen.merged_frame))

In [ ]:
fam_th = 0.1
%time fams = ab_fam_gen.run(fam_th, 30, compute_translator=False)

In [ ]:
out_dir = 'abr_fams'
for i in range(len(fams)):
    fams[i].sort_values(['counts'], ascending=[False]).to_csv(out_dir+'/'+ids[i]+'_abr_fams.tsv', sep='\t')

In [ ]:
fams[7].sort_values('counts')